In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('http://bit.ly/kaggletrain')

In [20]:
df.shape

(891, 12)

In [21]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [22]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [23]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
df = df.loc[df.Embarked.notna(), ['Survived', 'Pclass', 'Sex', 'Embarked']]

# selecting all rows except the ones having NAN in Embarked Column

In [25]:
df.shape

(889, 4)

In [26]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [27]:
X = df.loc[:, ['Pclass']]
y = df.Survived

In [28]:
# Both the above arguments are slightly different

X.shape

(889, 1)

In [29]:
y.shape 
# this gives 1 dimensional shape
#, but it is always preferred to have a 2 dimensional matrix to avoid errors

(889,)

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
logreg = LogisticRegression()

In [32]:
from sklearn.model_selection import cross_val_score

In [33]:
cross_val_score(logreg, X, y, cv = 5, scoring='accuracy').mean()

0.6783406335301212

<div class='alert alert-info'>
    Cross-validation is a technique for validating the model efficiency by training it on the subset of input data and testing on previously unseen subset of the input data.
</div>

In [34]:
y.value_counts(normalize = True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

# ENCODING

In [46]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False) 

In [49]:
ohe.fit_transform(df[['Sex']])
# Notice that we need to pass 2D array in fit_transform 
#of one hot encoder

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [48]:
ohe.categories_

[array(['female', 'male'], dtype=object)]

In [52]:
x = df.drop('Survived', axis='columns')
x.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [55]:
from sklearn.compose import make_column_transformer

# It is used when we have features in our dataframe which need 
# Different preprocessing.

# for example, we apply OneHotEncoding to only
#few columns out of our working DF

In [57]:
column_trans = make_column_transformer((OneHotEncoder(), ['Sex', 'Embarked']),
                                     remainder = 'passthrough')

# apply OneHotEncoder to given columns and 
# Passthrough the remaining columns

In [59]:
column_trans.fit_transform(x)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

<div class='alert alert-info'>
    We can add more to make_column_transformer, like we can add simple imputer to few other columns and Passthrough the rest of the columns.
</div>

# PIPELINES

In [60]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(column_trans, logreg)

<div class='alert alert-info'>
    <b>
        The use of pipeline is that, it conducts all the preprocessing along with the model implementation altogether.
        Hence, If we want to train a model with different dataset, where the structure is same, we can directly use the pipeline for training.
    </b>
</div>

In [61]:
cross_val_score(pipe, x, y, cv=5, scoring='accuracy').mean()

0.7727924839713071

**HENCE OUR MODEL ACCURACY WENT UP FROM 67% to 77%**

##  <font color='purple'>Testing the pipeline on different sample</font>

In [63]:
x_new = x.sample(5, random_state=99)
x_new

,Pclass,Sex,Embarked
599,1,male,C
512,1,male,S
273,1,male,C
215,1,female,C
790,3,male,Q


In [64]:
pipe.fit(x,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [65]:
pipe.predict(x_new)

array([1, 0, 1, 1, 0])

<div class='alert alert-info'>
The above model worked without preprocessing because it is already handled in our pipeline
</div>

<div class='alert alert-success'>
<h2>Advantages of using this approach</h2>
<ul>
    <li>We could have used get dummies method of Pandas on this model, but we would have to do the same on the test dataset as well. but suppose for a categorical column, train dataset has more categories within a column compared to that in the test dataset, then the dimensions of both the datasets would be different, and hence can cause problems.</li>
    
<li> The method of cross validation of scikit-learn applied on a model, preprocessed by functions of the other libraries becomes less reliable.</li>
<li>You don't need to create a Gigantic dataframe. Like here, OneHotEncoder did not change the shape of the dataframe.</li>
<li>We can conduct <b>grid search</b> on X and Y parameters of both X and Y datasets</li>
<ul>
</div>

<font color="red"> <b>GridSearchCV</b> is a technique to search through the best parameter values from the given set of the grid of parameters. It is basically a cross-validation method. the model and the parameters are required to be fed in. Best parameter values are extracted and then the predictions are made.</font>